In [ ]:
import requests, json
import pandas as pd
from datetime import datetime

url = "https://api-op.grid.gg/central-data/graphql"
headers = {"x-api-key": ""}

# Valorant ID - 6
# LOL ID - 3
# gte: "2025-01-01T15:00:07+02:00"
# lte: "2026-01-20T15:00:07+02:00" 

# Get Tournaments 
### (issue not match with series tournament IDs)

In [80]:
tournaments_query = """query GetTournaments($first: Int, $after: String) {
  tournaments (
  first: $first, 
    after: $after,
    filter : {
    title: { id: { in: ["6"] } },
       startDate: {
        gte: "2024-01-01" 
		    lte: "2026-01-20"
      }
  }) {
    pageInfo {
      hasPreviousPage
      hasNextPage
      startCursor
      endCursor
    }
    totalCount
    edges {
      cursor
      node {
        ...tournamentFields
      }
    }
  }
}
fragment tournamentFields on Tournament {
  id
  name
  nameShortened
  startDate
  endDate
  venueType
  titles{
    id
    name
  }
  teams{
    name
    id
  }
  children{
    id
    name
  }
}"""

all_tournaments = []
has_next = True
cursor = None  # Start with no cursor
limit = 10     # How many results per page

while has_next:
    # Pass variables into the JSON payload
    variables = {
        "first": limit,
        "after": cursor
    }
    payload = {
    "query": tournaments_query,
    "variables": variables
}
    response = requests.post(
        url, 
        json={'query': tournaments_query, 'variables': variables}, 
        headers=headers
    )
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        result = data['data']['tournaments']
        
        # 1. Add current page nodes to our list
        all_tournaments.extend([edge['node'] for edge in result['edges']])
        
        # 2. Update pagination info
        has_next = result['pageInfo']['hasNextPage']
        cursor = result['pageInfo']['endCursor']
        
        print(f"Fetched {len(result['edges'])} items. Next page available: {has_next}")
    else:
        print(f"Error: {response.text}")
        break

print(f"Total tournaments collected: {len(all_tournaments)}")

{'data': {'tournaments': {'pageInfo': {'hasPreviousPage': False, 'hasNextPage': False, 'startCursor': 'PjMLC1oLC0BCQERARgsLWgsL', 'endCursor': 'PjMLC1oLC09FQUFBRwsLWgsL'}, 'totalCount': 7, 'edges': [{'cursor': 'PjMLC1oLC0BCQERARgsLWgsL', 'node': {'id': '757371', 'name': 'VCT Americas - Kickoff 2024', 'nameShortened': 'Kickoff 2024', 'startDate': '2024-02-15', 'endDate': '2024-03-04', 'venueType': 'LAN', 'titles': [{'id': '6', 'name': 'Valorant'}], 'teams': [{'name': 'NRG', 'id': '97'}, {'name': 'FURIA', 'id': '99'}, {'name': 'Cloud9', 'id': '79'}, {'name': 'MIBR (1)', 'id': '81'}, {'name': 'LOUD (1)', 'id': '3412'}, {'name': 'Sentinels', 'id': '1079'}, {'name': 'Leviatán Esports', 'id': '1611'}, {'name': '100 Thieves', 'id': '337'}, {'name': 'KRÜ Esports', 'id': '48457'}, {'name': 'G2 Esports', 'id': '96'}, {'name': 'Evil Geniuses', 'id': '281'}], 'children': [{'id': '757372', 'name': 'VCT Americas - Kickoff 2024 (Groups)'}, {'id': '757581', 'name': 'VCT Americas - Kickoff 2024 (Playof

In [86]:
def tournaments_to_dataframe(all_tournaments):
    rows = []

    for t in all_tournaments:
        title = t["titles"][0] if t.get("titles") else {}

        teams = t.get("teams", [])
        team_ids = [team["id"] for team in teams if "id" in team]

        rows.append({
            "tournament_id": t.get("id"),
            "tournament_name": t.get("name"),
            "start_date": t.get("startDate"),
            "end_date": t.get("endDate"),
            "venue_type": t.get("venueType"),
            "title_id": title.get("id"),
            "title_name": title.get("name"),
            "children": json.dumps(t.get('children', [])),
            "num_teams": len(teams),
            "teams_id": team_ids,
        })

    df = pd.DataFrame(rows)

    # Convert the list column to a string before saving to SQL
    df['teams_id'] = df['teams_id'].apply(lambda x: ','.join(map(str, x)))

  # Convert to datetime for timeline logic
    df["start_date"] = pd.to_datetime(df["start_date"])
    df["start_date"] = df["start_date"].dt.strftime('%Y-%m-%d')
    df["end_date"] = pd.to_datetime(df["end_date"])
    df["end_date"] = df["end_date"].dt.strftime('%Y-%m-%d')

    # Sort chronologically (important for timeline models)
    df = df.sort_values("start_date").reset_index(drop=True)

    return df
df_tournaments = tournaments_to_dataframe(all_tournaments)
df_tournaments

,tournament_id,tournament_name,start_date,end_date,venue_type,title_id,title_name,children,num_teams,teams_id
0,757371,VCT Americas - Kickoff 2024,2024-02-15,2024-03-04,LAN,6,Valorant,"[{""id"": ""757372"", ""name"": ""VCT Americas - Kick...",11,"97,99,79,81,3412,1079,1611,337,48457,96,281"
1,757614,VALORANT Masters - Masters Madrid,2024-03-14,2024-03-24,LAN,6,Valorant,"[{""id"": ""757615"", ""name"": ""VCT Masters - Maste...",8,"3412,1079,173,3554,404,917,361,359"
2,757481,VCT Americas - Stage 1 2024,2024-04-06,2024-05-12,LAN,6,Valorant,"[{""id"": ""757482"", ""name"": ""VCT Americas - Stag...",11,"1079,79,96,48457,97,1611,3412,337,81,99,281"
3,774782,VCT Americas - Stage 2 2024,2024-06-16,2024-07-22,LAN,6,Valorant,"[{""id"": ""774783"", ""name"": ""VCT Americas - Stag...",11,"1611,337,3412,281,99,81,48457,79,96,1079,97"
4,775516,VCT Americas - Kickoff 2025,2025-01-16,2025-02-08,LAN,6,Valorant,"[{""id"": ""775517"", ""name"": ""VCT Americas - Kick...",12,"281,3412,81,337,53367,99,97,79,48457,1079,1611,96"
5,800675,VCT Americas - Stage 1 2025,2025-03-21,2025-05-04,LAN,6,Valorant,"[{""id"": ""800676"", ""name"": ""VCT Americas - Stag...",12,"281,3412,81,337,53367,99,97,79,48457,1079,1611,96"
6,826660,VCT Americas - Stage 2 2025,2025-07-18,2025-09-01,LAN,6,Valorant,"[{""id"": ""826661"", ""name"": ""VCT Americas - Stag...",12,"281,3412,81,337,53367,99,97,79,48457,1079,1611,96"


In [87]:
df_tournaments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tournament_id    7 non-null      object
 1   tournament_name  7 non-null      object
 2   start_date       7 non-null      object
 3   end_date         7 non-null      object
 4   venue_type       7 non-null      object
 5   title_id         7 non-null      object
 6   title_name       7 non-null      object
 7   children         7 non-null      object
 8   num_teams        7 non-null      int64 
 9   teams_id         7 non-null      object
dtypes: int64(1), object(9)
memory usage: 692.0+ bytes


In [88]:
import pandas as pd

from sqlalchemy import create_engine
# 'sqlite:///filename.db' for local file
engine = create_engine('sqlite:///valorant_esports.db')

df_tournaments.to_sql('tournaments', con=engine, if_exists='replace')

7

# Series Data

In [41]:
series_query = """
query GetAllSeries($first: Int, $after: String) {
  allSeries(
  first: $first, 
    after: $after,
    filter:{
      titleId: 6,
      startTimeScheduled:{
        gte: "2024-01-01T15:00:07+02:00"
        lte: "2026-01-20T15:00:07+02:00"
      }
    }
    orderBy: StartTimeScheduled    
  ) {
    totalCount,
    pageInfo{
      hasPreviousPage
      hasNextPage
      startCursor
      endCursor
    }
    edges{
      cursor
      node{
        ...seriesFields
      }
    }
  }
}
fragment seriesFields on Series {
  id
  title {
    name
  }
  type
  tournament {
    id
    name
    nameShortened
  }
  players{
    id
    nickname    
  }
  startTimeScheduled
  format {
    name
    nameShortened
  }
  teams {
    baseInfo {
      name
      id
    }    
    scoreAdvantage
  }
}""" 

all_series = []
has_next = True
cursor = None  # Start with no cursor
limit = 50    # How many results per page

while has_next:
    # Pass variables into the JSON payload
    variables = {
        "first": limit,
        "after": cursor
    }
    payload = {
    "query": series_query,
    "variables": variables
}
    response = requests.post(
        url, 
        json={'query': series_query, 'variables': variables}, 
        headers=headers
    )
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        result = data['data']['allSeries']
        
        # 1. Add current page nodes to our list
        all_series.extend([edge['node'] for edge in result['edges']])
        
        # 2. Update pagination info
        has_next = result['pageInfo']['hasNextPage']
        cursor = result['pageInfo']['endCursor']
        
        print(f"Fetched {len(result['edges'])} items. Next page available: {has_next}")
    else:
        print(f"Error: {response.text}")
        break

print(f"Total series collected: {len(all_series)}")

{'data': {'allSeries': {'totalCount': 212, 'pageInfo': {'hasPreviousPage': False, 'hasNextPage': True, 'startCursor': 'JAMWBQMjHhoSJBQfEhMCGxITCwtaCwtFQUVORE5HCwtaCwtFR0VDWkdFWkZBI0VFTUdHTUdHLQ==', 'endCursor': 'JAMWBQMjHhoSJBQfEhMCGxITCwtaCwtFQUJETk9DCwtaCwtFR0VDWkdDWkVFI0VGTUdHTUdHLQ=='}, 'edges': [{'cursor': 'JAMWBQMjHhoSJBQfEhMCGxITCwtaCwtFQUVORE5HCwtaCwtFR0VDWkdFWkZBI0VFTUdHTUdHLQ==', 'node': {'id': '2629390', 'title': {'name': 'Valorant'}, 'type': 'ESPORTS', 'tournament': {'id': '757073', 'name': 'VCT Americas - Kickoff 2024 (Groups: Group A)', 'nameShortened': 'Group A'}, 'players': [], 'startTimeScheduled': '2024-02-16T22:00:00Z', 'format': {'name': 'best-of-3', 'nameShortened': 'Bo3'}, 'teams': [{'baseInfo': {'name': 'FURIA', 'id': '99'}, 'scoreAdvantage': 0}, {'baseInfo': {'name': 'NRG', 'id': '97'}, 'scoreAdvantage': 0}]}}, {'cursor': 'JAMWBQMjHhoSJBQfEhMCGxITCwtaCwtFQUVORE5GCwtaCwtFR0VDWkdFWkZAI0dGTUdHTUdHLQ==', 'node': {'id': '2629391', 'title': {'name': 'Valorant'}, 'type

In [46]:
import json
# Save to a file
with open("all_series.json", "w") as f:
    json.dump(all_series, f, indent=4)


In [67]:
import pandas as pd

def series_to_dataframe(all_series: list) -> pd.DataFrame:
    rows = []

    for edge in all_series:
        s = edge
        # Tournament
        tournament = s.get("tournament", {}) or {}

        # Players
        players = s.get("players", [])
        player_ids = [p["id"] for p in players if "id" in p]

        # Teams
        teams = s.get("teams", [])
        team_ids = [
            t["baseInfo"]["id"]
            for t in teams
            if t.get("baseInfo") and "id" in t["baseInfo"]
        ]

        rows.append({
            "series_id": s.get("id"),

            # "game_name": s.get("title", {}).get("name"),
            "game_id": 6, # valorant ID
            "series_type": s.get("type"),
            "format_name": s.get("format", {}).get("name"),
            "format_short": s.get("format", {}).get("nameShortened"),

            # Tournament linkage
            "tournament_id": tournament.get("id"),
            "tournament_name": tournament.get("name"),

            # Temporal
            "start_date": s.get("startTimeScheduled"),

            # AI signals
            "player_ids": player_ids,
            "team_ids": team_ids,
            "num_players": len(player_ids),
            "num_teams": len(team_ids),
        })

    df = pd.DataFrame(rows)

    df["player_ids"] = df["player_ids"].apply(lambda x: ",".join(x))
    df["team_ids"] = df["team_ids"].apply(lambda x: ",".join(x))

    # Convert datetime for timeline reasoning 
    df["start_date"] = pd.to_datetime(df["start_date"])
    df["start_date"] = df["start_date"].dt.strftime('%Y-%m-%d')


    return df

df_series = series_to_dataframe(all_series)
df_series.head()

,series_id,game_id,series_type,format_name,format_short,tournament_id,tournament_name,start_date,player_ids,team_ids,num_players,num_teams
0,2629390,6,ESPORTS,best-of-3,Bo3,757073,VCT Americas - Kickoff 2024 (Groups: Group A),2024-02-16,,"99,97",0,2
1,2629391,6,ESPORTS,best-of-3,Bo3,757073,VCT Americas - Kickoff 2024 (Groups: Group A),2024-02-17,,"81,79",0,2
2,2629392,6,ESPORTS,best-of-3,Bo3,757628,VCT Americas - Kickoff 2024 (Groups: Group B),2024-02-17,,"1079,3412",0,2
3,2629393,6,ESPORTS,best-of-3,Bo3,757628,VCT Americas - Kickoff 2024 (Groups: Group B),2024-02-18,,"337,1611",0,2
4,2629394,6,ESPORTS,best-of-3,Bo3,757629,VCT Americas - Kickoff 2024 (Groups: Group C),2024-02-18,,"96,48457",0,2


In [68]:
import pandas as pd

from sqlalchemy import create_engine
# 'sqlite:///filename.db' for local file
engine = create_engine('sqlite:///valorant_esports.db')

df_series.to_sql('series', con=engine, if_exists='replace')

212

## DB player analysis

In [3]:
import sqlite3
import pandas as pd

# 1. Connect to the database
conn = sqlite3.connect('valorant_esports.db')

# 2. Use a SQL query to fetch data directly into a DataFrame
# query = "SELECT * FROM series"
query = "SELECT * FROM series WHERE player_ids != ''"
df = pd.read_sql_query(query, conn)

# 3. Close the connection
conn.close()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            23 non-null     int64 
 1   series_id        23 non-null     object
 2   game_id          23 non-null     int64 
 3   series_type      23 non-null     object
 4   format_name      23 non-null     object
 5   format_short     23 non-null     object
 6   tournament_id    23 non-null     object
 7   tournament_name  23 non-null     object
 8   start_date       23 non-null     object
 9   player_ids       23 non-null     object
 10  team_ids         23 non-null     object
 11  num_players      23 non-null     int64 
 12  num_teams        23 non-null     int64 
dtypes: int64(4), object(9)
memory usage: 2.5+ KB


In [13]:
df['series_id'].value_counts()

series_id
2819694    1
2819695    1
2819696    1
2819698    1
2819699    1
2819700    1
2819701    1
2819702    1
2819703    1
2819704    1
2819705    1
2843060    1
2843061    1
2843062    1
2843063    1
2843064    1
2843065    1
2843067    1
2843066    1
2843068    1
2843069    1
2843070    1
2843071    1
Name: count, dtype: int64

In [4]:
import pandas as pd

df_expanded = df.assign(team_ids=df['team_ids'].str.split(',')).explode('team_ids')

# 2. Group by the Team ID and aggregate 
analysis_df = df_expanded.groupby('team_ids').agg({
    'team_ids': 'count',
    'player_ids': lambda x: list(set(','.join(filter(None, x)).split(',')))
}).rename(columns={'team_ids': 'occurrence_count', 'player_ids': 'associated_players'}).reset_index()

# 2. Add a column that calculates the total count of unique players for each team
analysis_df['player_count'] = analysis_df['associated_players'].apply(len)

# 3. Sort by occurrence to see the top teams first
analysis_df = analysis_df.sort_values(by='occurrence_count', ascending=False).reset_index(drop=True)

# Display the results
analysis_df

,team_ids,occurrence_count,associated_players,player_count
0,97,7,"[1193, 10612, 206, 10901, 77, 11013, 297, 3078...",40
1,79,6,"[2345, 1193, 10612, 10901, 11013, 297, 1192, 2...",34
2,96,6,"[2345, 1193, 10612, 10901, 11013, 297, 1192, 2...",38
3,1079,5,"[1193, 10612, 11013, 10901, 297, 1192, 74432, ...",33
4,337,5,"[2345, 1193, 10612, 10901, 1192, 2359, 307, 33...",29
5,1611,4,"[2345, 1193, 10612, 206, 10901, 77, 297, 3078,...",29
6,281,3,"[2345, 11013, 2359, 307, 42759, 2341, 866, 42,...",20
7,3412,2,"[2837, 2345, 119048, 629, 2361, 296, 32038, 23...",12
8,53367,2,"[10636, 287, 94, 1193, 10612, 1686, 10901, 289...",17
9,48457,2,"[2513, 206, 77, 297, 3078, 28813, 74, 2358, 74...",11


In [5]:
exploded_players = df['player_ids'].dropna().str.split(',').explode()

exploded_players = exploded_players.str.strip()
exploded_players = exploded_players[exploded_players != '']

player_stats_df = exploded_players.value_counts().reset_index()

player_stats_df.columns = ['player_id', 'appearance_count']

player_stats_df = player_stats_df.sort_values(by='appearance_count', ascending=False).reset_index(drop=True)

player_stats_df

,player_id,appearance_count
0,2358,7
1,74432,7
2,297,7
3,2513,7
4,2340,7
5,10636,6
6,1193,6
7,272,6
8,1195,6
9,10612,6


In [12]:
# 1. Split and explode the team_ids while keeping other IDs attached
# We include series_id and tournament_id here
df_expanded = df.assign(team_ids=df['team_ids'].str.split(',')).explode('team_ids')

# 2. Updated Aggregation: Collect unique Series and Tournament IDs
analysis_df = df_expanded.groupby('team_ids').agg({
    'team_ids': 'count',
    'player_ids': lambda x: list(set(','.join(filter(None, x)).split(','))),
    'series_id': lambda x: list(set(x.dropna())),      # Unique series IDs for this team
    'tournament_id': lambda x: list(set(x.dropna()))  # Unique tournament IDs
}).rename(columns={'team_ids': 'occurrence_count', 'player_ids': 'associated_players'}).reset_index()

# 3. Add the player count column
analysis_df['player_count'] = analysis_df['associated_players'].apply(len)

# 4. Filter for Veteran Teams (using your top_10_player_ids from the previous step)
top_10_player_ids = player_stats_df.head(12)['player_id'].tolist()
has_top_player = lambda players: any(p in top_10_player_ids for p in players)

veteran_teams_df = analysis_df[analysis_df['associated_players'].apply(has_top_player)].copy()

# 5. Add the summary of which top players are in each team
veteran_teams_df['top_players_present'] = veteran_teams_df['associated_players'].apply(
    lambda x: [p for p in x if p in top_10_player_ids]
)

# Sort for the most active veteran teams
veteran_teams_df = veteran_teams_df.sort_values(by='occurrence_count', ascending=False).reset_index(drop=True)
veteran_teams_df['series_count']  = veteran_teams_df['series_id'].count()
veteran_teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   team_ids             9 non-null      object
 1   occurrence_count     9 non-null      int64 
 2   associated_players   9 non-null      object
 3   series_id            9 non-null      object
 4   tournament_id        9 non-null      object
 5   player_count         9 non-null      int64 
 6   top_players_present  9 non-null      object
 7   series_count         9 non-null      int64 
dtypes: int64(3), object(5)
memory usage: 708.0+ bytes


In [10]:
veteran_teams_df['top_players_present'].explode().value_counts()

top_players_present
1193      7
10612     7
10901     7
297       7
3301      7
74432     7
2340      7
10636     7
2358      7
2513      7
11338     7
2890      7
277       7
2348      7
1686      7
1195      7
272       7
1192      6
119048    6
296       6
2359      6
2341      6
2361      6
11013     6
2351      6
287       6
709       6
94        6
2349      6
2353      6
727       6
2347      6
2345      6
2512      6
303       4
866       4
306       4
307       4
14940     4
3320      3
14        3
32038     3
2837      3
10630     3
208       3
629       3
42759     3
48        3
42        3
32030     3
Name: count, dtype: int64

In [11]:
veteran_teams_df['series_id'].explode().value_counts()

series_id
2843069    2
2843071    2
2843070    2
2843063    2
2819700    2
2843067    2
2819695    2
2819704    2
2843062    2
2843066    2
2819694    2
2843060    2
2843068    2
2819698    2
2843061    2
2819696    2
2819703    2
2843065    2
2819699    2
2819701    2
2843064    2
2819702    2
2819705    2
Name: count, dtype: int64